In [1]:
import os
import shutil
import glob
import types
import MontagePy.main as m
from functools import wraps

In [2]:
class MontageError(Exception):
    pass

def decorate_montage(module, decorator):
    for name in dir(module):
        obj = getattr(module, name)
        if isinstance(obj, types.BuiltinFunctionType):
            setattr(module, name, decorator(obj))

def montage_exception(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        rtn = func(*args, **kwargs)
        if rtn['status'] == '1':
            raise MontageError(rtn['msg'].decode('utf-8'))
        else:
            print(func.__name__, rtn)
    return wrapper

decorate_montage(m, montage_exception)

In [3]:
def montage_mosaic(tiles, out_file, quick_proj=False, coadd_type='average', correct_bg=True, workdir=None,
                   clean_workdir=True):
    """
    Automates the Montage mosaicking process on the input tiles.

    Parameters
    ----------
    tiles : list-like
        Input images to be mosaicked.
    out_file : str
        File name of the output mosaic image.
    quick_proj : bool, optional
        Flag to use the Quick Look projection method. Defaults to `False`.
    coadd_type : {'average', 'median', 'sum'}, optional
        Defines the coaddition type in stacking. Defaults to `'average'`.
    correct_bg : bool, optional
        Determines if we should background correct the images before coadding. Defaults to `True`.
    workdir : str, optional
        Name for temporary work directory. If not provided defaults to `'montage_workdir'`.
    clean_workdir : bool, optional
        Removes the temporary work directory structure once complete. Defaults to `True`.

    """

    coadd_dict = {'average': 0,
                  'median': 1,
                  'sum': 2}

    # Build the temporary work directory structure
    if workdir is None:
        workdir = 'montage_workdir'

    raw_dir = workdir + '/raw'
    projected_dir = workdir + '/projected'
    diff_dir = workdir + '/diffs'
    corrected_dir = workdir + '/corrected'

    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(projected_dir, exist_ok=True)
    os.makedirs(diff_dir, exist_ok=True)
    os.makedirs(corrected_dir, exist_ok=True)

    # Symlink the files into the raw directory
    for tile in tiles:
        os.symlink(tile, os.path.join(raw_dir, os.path.basename(tile)))
#         shutil.copy(tile, raw_dir)

    # Generate the metatable for the raw images
    raw_metatable = workdir + '/rImages.tbl'
    m.mImgtbl(raw_dir, raw_metatable)

    # Create the region header to cover the mosaic area
    region_hdr = workdir + '/region.hdr'
    m.mMakeHdr(raw_metatable, region_hdr)

    # Perform reprojectioning
    m.mProjExec(raw_dir, raw_metatable, region_hdr, projdir=projected_dir, quickMode=quick_proj)

    # Generate the metatable for the projected images
    projected_metatable = workdir + '/pImages.tbl'
    m.mImgtbl(projected_dir, projected_metatable)

    if correct_bg:
        # Determine overlaps between the tiles
        diff_table = workdir + '/diffs.tbl'
        m.mOverlaps(projected_metatable, diff_table)

        # Generate the difference images and fit them
        diff_fit_table = workdir + '/fits.tbl'
        m.mDiffFitExec(projected_dir, diff_table, region_hdr, diff_dir, diff_fit_table)

        # Model the background corrections
        corrections_table = workdir + '/corrections.tbl'
        m.mBgModel(projected_metatable, diff_fit_table, corrections_table)

        # Background correct the projected images
        m.mBgExec(projected_dir, projected_metatable, corrections_table, corrected_dir)

        # Create the metatable for the corrected images
        corrected_metatable = workdir + '/cImages.tbl'
        m.mImgtbl(corrected_dir, corrected_metatable)

        # Coadd the background-corrected, projected images
        m.mAdd(corrected_dir, corrected_metatable, region_hdr, out_file)

    else:
        # Coadd the projected images without background corrections
        m.mAdd(projected_dir, projected_metatable, region_hdr, out_file, coadd=coadd_dict[coadd_type])

    # Clean up the work directory
    if clean_workdir:
        shutil.rmtree(workdir)

In [4]:
tiles = glob.glob('SPT_AGN/Data/SPTPol/images/ssdf_tiles/I1_SSDF4.*_mosaic.fits')
tiles = [os.path.abspath(tile) for tile in tiles if '4.0' in tile or '4.1' in tile]
tiles

['/lustre/work/mei/bfloyd/SPT_AGN/Data/SPTPol/images/ssdf_tiles/I1_SSDF4.0_mosaic.fits',
 '/lustre/work/mei/bfloyd/SPT_AGN/Data/SPTPol/images/ssdf_tiles/I1_SSDF4.1_mosaic.fits']

In [5]:
montage_mosaic(tiles, 'montage_test_conserved.fits', correct_bg=False)

mImgtbl {'status': '0', 'count': 2, 'badfits': 0, 'badwcs': 0}
mMakeHdr {'status': '0', 'count': 2, 'ncube': 0, 'naxis1': 14705, 'naxis2': 13955, 'clon': 358.7904474840388, 'clat': -50.86220132116989, 'lonsize': 2.450838235, 'latsize': 2.325837985, 'posang': 0.22720678363638785, 'lon1': 0.7880122828242975, 'lat1': -52.00319877001041, 'lon2': 356.8074255480142, 'lat2': -52.013160146173334, 'lon3': 356.8896325290504, 'lat3': -49.688751936431906, 'lon4': 0.6766364871069754, 'lat4': -49.679274825650424}
mProjExec {'status': '0', 'count': 2, 'failed': 0, 'nooverlap': 0}
mImgtbl {'status': '0', 'count': 2, 'badfits': 0, 'badwcs': 0}


MontageError: b'error reading from FITS file'

In [ ]:
from astropy.io import fits
from astropy.visualization import imshow_norm, ZScaleInterval
import matplotlib.pyplot as plt
%matplotlib inline

final_image = fits.getdata('montage_test_conserved.fits')
imshow_norm(final_image, origin='lower', cmap='gray_r', interval=ZScaleInterval())
plt.show()